In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mutagen.mp3 import MP3
from pydub import AudioSegment
import random

from threading import Thread
from time import sleep

from tqdm.notebook import tqdm

load_dotenv("../.env")

True

In [2]:
VALIDATED_LIST_PATH = os.path.join(os.getenv("CLIPS_META_PATH"), "validated.tsv")
X_VECTOR_SAMPLES_PATH = os.path.join(os.getenv("X_VECTOR_SAMPLES_PATH"))

In [3]:
# Load clips metadata
clips_meta = pd.read_table(VALIDATED_LIST_PATH)

In [4]:
CLIPS_CROSSFADE_DURATION = 100 # ms
MAX_CLIP_DURATION = 30 # seconds
THREADS_COUNT = 24

# Get all unique in client_id column
client_ids = list(clips_meta["client_id"].unique())

processed_counter = 0

In [5]:
def process_client():
    global processed_counter

    while True:
        try:
            client_id = client_ids.pop(0)
        except:
            break

        client_clips = clips_meta[clips_meta.client_id == client_id]

        clips_paths = [os.path.join(os.getenv("CLIPS_PATH"), clip_path + ".mp3") for clip_path in client_clips["path"]]
        random.shuffle(clips_paths)

        output_length = 0
        audio_segments = []

        for clip in clips_paths:
            current_segment = AudioSegment.from_mp3(clip)

            if output_length + current_segment.duration_seconds > MAX_CLIP_DURATION:
                break
            
            output_length += current_segment.duration_seconds
            audio_segments.append(current_segment)

        # Skip clients with less than 2 seconds of audio
        if output_length < 2:
            continue
        
        output_clip = audio_segments.pop(0)
        for audio_segment in audio_segments:
            output_clip = output_clip.append(audio_segment, crossfade=CLIPS_CROSSFADE_DURATION)

        output_clip.export(os.path.join(X_VECTOR_SAMPLES_PATH, f"{client_id}.wav"), format="wav")

        processed_counter += 1

In [6]:
processed_counter = 0
last_count = processed_counter

thread_handles = [Thread(target=process_client, daemon=True) for _ in range(THREADS_COUNT)]

with tqdm(total=len(client_ids)) as pbar:
    for thread in thread_handles:
        thread.start()

    while len(client_ids) > 0:
        tmp = processed_counter
        update_value = tmp - last_count
        
        pbar.update(update_value)
        pbar.refresh()

        last_count = tmp
        sleep(0.1)

for thread in thread_handles:
    thread.join()

  0%|          | 0/26744 [00:00<?, ?it/s]